In [1]:
%load_ext autotime
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# Set this to False to rebuild filelist.txt and meta.csv
use_cache = False

In [36]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if use_cache and os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Archive/Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

  0%|          | 0/4 [00:00<?, ?it/s]

0                                                                              Archive/Gabrielle/
1                                                                        Archive/Gabrielle/Orders
2                                                                   Archive/Gabrielle/Orders/AOIs
3                                 Archive/Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                      Archive/Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                                   ...                                           
55495    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L02_R00000062_C00000065.tfw
55496    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L03_R00000020_C00000022.tfw
55497    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000125_C00000131.tif
55498    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000127_C00000133.tif
55499               

In [37]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29912,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
29913,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp
29916,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp
29917,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp
29924,Archive/Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp
...,...
112081,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
112086,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
113187,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
113188,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_13FEB2021.shp


In [38]:
image_extensions = (".jpg", ".jp2", ".tif", ".TIF", ".png")

In [39]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith(image_extensions)]
images

1084      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
1085      MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
1087        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
1090        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
1104        MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
113420            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
113426            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
113429            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
113430            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
113433            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Length: 2650, dtype: object

In [40]:
Gabrielle_images = filelist[filelist.str.contains("Gabrielle") & filelist.str.endswith(image_extensions)]
Gabrielle_images

289                                                                                                                                                    Archive/Gabrielle/Pressure sensors/Emma's field notes.jpg
314                                                                                                                  Archive/Gabrielle/Imagery/skyfiworking/SkyFi_4737_Northland_New-Zealand_2023-03-03_2250.png
315                                                                                                                  Archive/Gabrielle/Imagery/skyfiworking/SkyFi_4794_Northland_New-Zealand_2023-03-25_2229.tif
319                                                                                                                    Archive/Gabrielle/Imagery/skyfiworking/SkyFi_4794_Waikato_New-Zealand_2023-08-25_2133.tif
320                                                                                                                    Archive/Gabrielle/Imagery/skyfiworking/SkyFi_

In [41]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    else:
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=images, processor=fuzz_preprocess)
        return match, score

df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))

/tmp/ipykernel_3722334/4132008065.py:24: TqdmWarning: Iterable length 2985 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))


  0%|          | 0/2985 [00:00<?, ?it/s]

Perfect matches: 1913
Imperfect matches: 1072


In [42]:
df = df.sort_values("match_score")
df

,filename,matched_image,match_score
30917,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054
30992,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529
30830,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061
33769,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111
29976,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111
...,...,...,...
48678,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14JAN2000_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14JAN2000_mosaic.jp2,100.000000
48685,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000
48689,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_beach.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000
48247,Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_16DEC1945_cliff.shp,Retrolens/Taranaki/PariokariwaPoint/Stack/PariokariwaPoint_16DEC1945_mosaic.jp2,100.000000


In [43]:
def get_shapefile_meta(tup):
    i, row = tup
    filename = row.filename
    source = "Unknown"
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return {"Source": source, "n_lines": 0}
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            source = "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            source = "MAX"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                source = "MAX"
        elif len(sources) == 1:
            source = sources[0]
        elif len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
            source = sources[0]
    n_lines = len(shapefile.dropna(subset="geometry").explode(index_parts=False))
    CPS = ""
    if "CPS" not in shapefile.columns:
        CPS = "No CPS column"
    else:
        CPS = shapefile.CPS.values
        if len(CPS) == 0:
            CPS = "No CPS values"
        elif 9 in CPS:
            CPS = "Has CPS=9"
        elif 0 in CPS:
            CPS = "Has CPS=0"
        elif None in CPS:
            CPS = "Has None CPS"
        elif pd.isna(CPS).any():
            CPS = "Has NA CPS"
        else:
            CPS = "OK"
    return {
        "Source": source,
        "n_lines": n_lines,
        "CPS": CPS,
        "Photoscale": "Photoscale" in shapefile.columns,
        "Pixel_Er": "Pixel_Er" in shapefile.columns
    }

shp_meta = pd.DataFrame(process_map(get_shapefile_meta, df.iterrows(), total=len(df)), index=df.index)
df = pd.concat([df, shp_meta], axis=1)
df

  0%|          | 0/2985 [00:00<?, ?it/s]

Retrolens/Waikato/OpitoBay/Shorelines/Opito_OtamaBay_14FEB2023.shp has ambiguous sources: ['PLE' 'MAX']
Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp has ambiguous sources: ['LDS' None]
Retrolens/Waikato/Whangamata/Shorelines/Whangamata_14FEB2023.shp has ambiguous sources: ['MAX' 'PLE']
Retrolens/Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Otago/Moeraki_HampdenBeach/Shorelines/Moeraki_HampdenBeach_24FEB2014.shp has ambiguous sources: ['LDS' None]
Retrolens/Northland/BayleysBeach/Shorelines/BayleysBeach_07APR2023.shp has ambiguous sources: ['MAX' None]
Retrolens/WestCoast/Westport/Shorelines/Westport_15JAN1988.shp has ambiguous sources: ['Rl' 'RL']
Retrolens/HawkesBay/Clifton_Haumoana/Shorelines/Clifton_Haumoana_23SEP1963.shp has ambiguous sources: ['RL' None]
MaxarImagery/HighFreq/Canterbury/PareoraRiver_Timaru/Sh

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30917,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054,MAX,6,OK,True,True
30992,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529,MAX,28,OK,True,True
30830,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061,MAX,24,OK,True,True
33769,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
29976,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
...,...,...,...,...,...,...,...,...
48678,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14JAN2000_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14JAN2000_mosaic.jp2,100.000000,RL,10,OK,True,True
48685,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000,RL,13,OK,True,True
48689,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_beach.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.000000,RL,8,OK,True,True
48247,Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_16DEC1945_cliff.shp,Retrolens/Taranaki/PariokariwaPoint/Stack/PariokariwaPoint_16DEC1945_mosaic.jp2,100.000000,RLN,27,OK,True,True


In [47]:
df.CPS.value_counts()

CPS
OK               2902
No CPS column      57
Has CPS=0          14
No CPS values      12
Name: count, dtype: int64

In [48]:
df[(df.CPS!="OK") & (df.n_lines > 0)][["filename", "Source", "n_lines", "CPS"]].sort_values(by=["CPS", "filename"]).to_csv(prefix+"Nick/CPS_issues.csv", index=False)

In [49]:
df.sort_values(by="match_score")

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30917,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054,MAX,6,OK,True,True
30992,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529,MAX,28,OK,True,True
30830,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061,MAX,24,OK,True,True
33769,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
29976,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
...,...,...,...,...,...,...,...,...
100066,Retrolens/Gisborne/TeAraroa/Shorelines/TeAraroa_1OCT1980.shp,Retrolens/Gisborne/TeAraroa/Stack/TeAraroa_1OCT1980_mosaic.jp2,100.000000,RLN,4,OK,True,True
100048,Retrolens/Gisborne/TeAraroa/Shorelines/TeAraroa_15MAY1983.shp,Retrolens/Gisborne/TeAraroa/Stack/TeAraroa_15MAY1983_mosaic.jp2,100.000000,RL,3,OK,True,True
88542,Retrolens/Northland/Rangiputa/Shorelines/Rangiputa_26MAY1947.shp,Retrolens/Northland/Rangiputa/Stack/Rangiputa_26MAY1947.tif,100.000000,RL,4,OK,True,True
88844,Retrolens/Northland/WhangareiHeads/Shorelines/WhangareiHeads_29JAN1969.shp,Retrolens/Northland/WhangareiHeads/Stack/WhangareiHeads_29JAN1969.tif,100.000000,RL,3,OK,True,True


In [50]:
failures = (df.match_score < 100) & (df.Source != "LDS") & (df.n_lines > 0)
print(sum(failures))
try:
  df[failures].to_csv(prefix+"Nick/failures.csv", index=False)
except:
  print("Can't write Nick/failures.csv")
df[failures]

562


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30917,Archive/Gabrielle/Shorelines/Northland/DoubtlessBay/DoubtlessBay_30AUG2022.shp,MaxarImagery/HighFreq/Tasman/Motueka/Imagery/Stack/Motueka_10AUG2003.tif,54.054054,MAX,6,OK,True,True
30992,Archive/Gabrielle/Shorelines/Northland/Ruakaka/Ruakaka_27DEC2022.shp,MaxarImagery/HighFreq/WestCoast/Punakaiki/Stack/Punakaiki_12FEB2023.tif,58.823529,MAX,28,OK,True,True
30830,Archive/Gabrielle/Shorelines/Northland/Great ExhibitionBay/Rarawa_11JULY2022.shp,MaxarImagery/HighFreq/Auckland/MuriwaiSouth/Stack/Muriwai_15JULY2017.tif,60.606061,MAX,24,OK,True,True
33769,Retrolens/Waikato/Whitianga/Shorelines/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
29976,Archive/Gabrielle/Shorelines/Waikato/Whitianga/Whitianga_18FEB2023.shp,MaxarImagery/HighFreq/Tasman/WainuiBay/Imagery/Stack/WainuiBay_11FEB2021.tif,61.111111,NEO,3,OK,True,True
...,...,...,...,...,...,...,...,...
96980,Retrolens/WestCoast/TaramakauRiver_North/Shorelines/TaramakauRiver_06DEC2022.shp,MaxarImagery/HighFreq/WestCoast/TaramakauRiverNorth/Stack/TaramakauRiverNorth_06DEC2022.tif,89.795918,MAX,8,OK,True,True
97137,Retrolens/WestCoast/Hokitika_South/Shorelines/Hokitika_South_28MAR2023.shp,MaxarImagery/HighFreq/WestCoast/HokitikaNorth/Stack/HokitikaNorth_28MAR2023.tif,90.909091,MAX,3,OK,True,True
34417,Retrolens/Waikato/PortWaikato/Shorelines/PortWaikato_29MAR2012.shp,MaxarImagery/HighFreq/Waikato/PortWaikato/Imagery/Stack/PortWaikato_09MAR2016.tif,92.307692,RL,1,OK,False,False
33642,Retrolens/Waikato/Mokau/Shorelines/Mokau_12JAN2013.shp,MaxarImagery/HighFreq/Waikato/Mokau/Imagery/Stack/Mokau_11JAN2013.tif,92.857143,RL,1,OK,False,False


In [51]:
df.to_csv(prefix+"Nick/shoreline_image_matching.csv", index=False)

### For those perfect matches, get the metadata from the corresponding matched image

In [52]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    try:
        image = rio.open(prefix + row.matched_image)
    except:
        print(f"Can't read {row.matched_image}")
        return {}
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read {row['filename']}")
        return {}
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if use_cache and os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

  0%|          | 0/1913 [00:00<?, ?it/s]

/home/nyou045/.local/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,interleave,GCPs,res,compress,photometric
0,Retrolens/Northland/NinetyMile2/Shorelines/NinetyMile2_21JAN2024.shp,MaxarImagery/HighFreq/Northland/NinetyMile2/Stack/NinetyMile2_21JAN2024.tif,100.0,NEO,7,True,True,True,GTiff,uint16,256.0,13592,14145,3,None,"(0.3000000000000069, 0.0, 1589451.5128915159, 0.0, -0.3, 6162740.502428755, 0.0, 0.0, 1.0)",128.0,128,True,pixel,4,"(0.3000000000000069, 0.3)",NaN,NaN
1,Retrolens/Gisborne/Wainui/Shorelines/Wainui_28SEP1977.shp,Retrolens/Gisborne/Wainui/Stack/Wainui_28SEP1977_mosaic.jp2,100.0,RL,4,True,True,True,JP2OpenJPEG,uint16,256.0,26962,21623,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.2793723223523756, 0.0, 2040674.326419808, 0.0, -0.2793723223523751, 5710244.673574584, 0.0, 0.0, 1.0)",NaN,21623,False,pixel,0,"(0.2793723223523756, 0.2793723223523751)",NaN,NaN
2,Retrolens/Canterbury/AshburtonRiver/Shorelines/AshburtonRiver_25FEB1976.shp,Retrolens/Canterbury/AshburtonRiver/Stack/AshburtonRiver_25FEB1976_mosaic.jp2,100.0,RL,9,True,True,True,JP2OpenJPEG,uint16,256.0,19400,12735,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.3, 0.0, 1502392.07258884, 0.0, -0.3, 5124384.541071977, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.3, 0.3)",NaN,NaN
3,Retrolens/Canterbury/AshburtonRiver/Shorelines/AshburtonRiver_31OCT1965.shp,Retrolens/Canterbury/AshburtonRiver/Stack/AshburtonRiver_31OCT1965_mosaic.jp2,100.0,RL,21,True,True,True,JP2OpenJPEG,uint16,256.0,15920,10570,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.6, 0.0, 1499150.8672684287, 0.0, -0.6, 5124561.36699031, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.6, 0.6)",NaN,NaN
4,Retrolens/Canterbury/AshburtonRiver/Shorelines/AshburtonRiver_31OCT1987.shp,Retrolens/Canterbury/AshburtonRiver/Stack/AshburtonRiver_31OCT1987_mosaic.jp2,100.0,RL,25,True,True,True,JP2OpenJPEG,uint16,256.0,10352,6915,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.9000000000000045, 0.0, 1499328.6291150467, 0.0, -0.9, 5124606.641994562, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(0.9000000000000045, 0.9)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14JAN2000_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14JAN2000_mosaic.jp2,100.0,RL,10,True,True,True,JP2OpenJPEG,uint16,256.0,8383,5309,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.3999999999999944, 0.0, 1684891.7222899452, 0.0, -1.3999999999999297, 5679544.697225339, 0.0, 0.0, 1.0)",1024.0,1024,True,pixel,0,"(1.3999999999999944, 1.3999999999999297)",NaN,NaN
1909,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_cliff.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.0,RL,13,True,True,True,JP2OpenJPEG,uint16,256.0,56494,32665,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.20000000000000082, 0.0, 1685313.7118251163, 0.0, -0.2, 5678879.025673549, 0.0, 0.0, 1.0)",NaN,32665,False,pixel,0,"(0.20000000000000082, 0.2)",NaN,NaN
1910,Retrolens/Taranaki/NewPlymouth/Shorelines/NewPlymouth_14AUG1950_beach.shp,Retrolens/Taranaki/NewPlymouth/Stack/NewPlymouth_14AUG1950_mosaic.jp2,100.0,RL,8,True,True,True,JP2OpenJPEG,uint16,256.0,56494,32665,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.20000000000000082, 0.0, 1685313.7118251163, 0.0, -0.2, 5678879.025673549, 0.0, 0.0, 1.0)",NaN,32665,False,pixel,0,"(0.20000000000000082, 0.2)",NaN,NaN
1911,Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_16DEC1945_cliff.shp,Retrolens/Taranaki/PariokariwaPoint/Stack/PariokariwaPoint_16DEC1945_mosaic.jp2,100.0,RLN,27,True,True,True,JP2OpenJPEG,uint16,256.0,13284,10478,3,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_

## Read LDS index tiles

In [53]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$")]
index_tiles

4525     Archive/Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10086            Archive/Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11964         Archive/Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13944         Archive/Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14063              Archive/Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
                                                                   ...                                                           
36387                     SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp
36392                SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-

In [54]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/79 [00:00<?, ?it/s]

147407

In [55]:
index_tiles.Captured = index_tiles.Captured.replace({"05, 06, 09, 10 Feb. 2014": "2014-02-05", "11 & 26 March 2014": "2014-03-11"})
index_tiles["parsed_date"] = pd.NA
for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:
    index_tiles["parsed_date"].update(pd.to_datetime(index_tiles[col].str.split(",|-|to").str[0], dayfirst=True, format="mixed", errors="ignore"))
index_tiles.parsed_date = index_tiles.parsed_date.astype("string")
index_tiles.parsed_date.value_counts(dropna=False)

/home/nyou045/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


parsed_date
2017                   23649
<NA>                   22679
2022                   11348
2023                    3607
24/02/2017              2685
                       ...  
9/11/2016                  1
2029-03-12 13:01:12        1
2005-05-12 12:05:22        1
2016-11-15 15:12:28        1
2028-11-15 16:03:27        1
Name: count, Length: 384, dtype: Int64

## LDS index tile matching

In [72]:
LDS = df[(df.Source == "LDS") & (df.match_score < 100)].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30738,Archive/Gabrielle/Shorelines/Hawkes Bay/Napier/Heretaunga_20FEB2023.shp,Retrolens/WestCoast/WhareateaRiver/Stack/WhareateaRiver_23FEB2023.tif,61.904762,LDS,9,OK,True,True
30279,Archive/Gabrielle/Shorelines/BayofPlenty/Papamoa/BOPLINZ_Papamoa_04APR2023.shp,MaxarImagery/HighFreq/BayOfPlenty/Ohiwa/Imagery/Stack/Ohiwa_08APR2013.tif,62.608696,LDS,7,OK,True,True
30207,Archive/Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,MaxarImagery/HighFreq/BayOfPlenty/Ohiwa/Imagery/Stack/Ohiwa_08APR2013.tif,62.608696,LDS,6,OK,True,True
30417,Archive/Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,MaxarImagery/HighFreq/Northland/CapeReinga/Stack/CapeReinga_24DEC2021.tif,63.157895,LDS,34,OK,True,True
102815,Retrolens/Gisborne/Waiparapara/Shorelines/Waiparapara_07MAR2023.shp,MaxarImagery/HighFreq/Gisborne/WaipiroBay/Imagery/Stack/WaipiroBay_22MAR2003.tif,63.157895,LDS,6,OK,True,True
...,...,...,...,...,...,...,...,...
31119,Archive/Gabrielle/Shorelines/Gisborne/TeAraroa/TeAraroa_19DEC2021.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_18DEC2021.tif,94.117647,LDS,11,OK,True,True
79611,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_01APR2006.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Ryans_Pipikaretu_Penguin_TeRauoneBeach_17MAR2000_mosaic.jp2,94.252874,LDS,8,OK,True,True
41375,Retrolens/Southland/Tiwai_Point/Shorelines/TiwaiPoint_07FEB2016.shp,MaxarImagery/HighFreq/Southland/TiwaiPoint/Stack/TiwaiPoint_02FEB2016.tif,94.444444,LDS,1,OK,True,True
49941,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_cliff.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Stack/Oakura_03OCT2016.tif,96.551724,LDS,5,OK,True,True


In [90]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured", "FLOWN1", "FLOWN2"]:#, "parsed_date"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  display(date, DSASdate, gdf, intersecting_tiles.dropna(axis=1, how="all"), date_options, date_to_col)
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    elif "southland-075m-rural-aerial-photos-index-tiles-2005-2011.shp" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .75
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "matched_date_col": col_for_match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
print("Perfect matches:", sum(LDS.match_score == 100))
print("Imperfect matches:", sum(LDS.match_score < 100))
print("Has Pixel_ER: ", sum(~LDS.Pixel_ER.isna()))
print("No Pixel_ER: ", sum(LDS.Pixel_ER.isna()))
LDS

  0%|          | 0/510 [00:00<?, ?it/s]

Perfect matches: 366
Imperfect matches: 144
Has Pixel_ER:  510
No Pixel_ER:  0


,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
68,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
231,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
178,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
131,Retrolens/Canterbury/OpihiRiver_South/Shorelines/OpihiRiver_South_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
286,Retrolens/Canterbury/RakaiaSouth/Shorelines/RakaiaSouth_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
...,...,...,...,...,...,...,...,...
185,Retrolens/Northland/TakouBay/Shorelines/TakouBay_09NOV2014.shp,[SpatialData/Mosaics/Footprint shapefiles/Northland Footrpints/northland-04m-rural-aerial-photos-index-tiles-2014-2016.shp],2014-11-09,9/11/2014,"14/03/2016, 09/11/2014",DATE_DMY,100.000000,0.4
184,Retrolens/Gisborne/HuataiBeach/Shorelines/HuataiBeach_16FEB2018.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-gisborne-03m-rural-aerial-photos-index-tiles-2017-2019-SHP (1)/gisborne-03m-rural-aerial-photos-index-tiles-2017-2019.shp],2018-02-16,16/02/2018,16/02/2018,FLOWN,100.000000,0.3
183,Retrolens/Manawatu/Waitarere/Shorelines/Waitarere_06JAN2016.shp,[SpatialData/Mosaics/Footprint shapefiles/Manawatu/manawatu-whanganui-03m-rural-aerial-photos-index-tiles-2015-.shp],2016-01-06,6/01/2016,06/01/2016,DATE_DMY,100.000000,0.3
198,Retrolens/Bay of Plenty/WaiotaheBeach/Shorelines/WaiotaheBeach_02JAN2022.shp,"[Archive/Gabrielle/Imagery/pre-storm/LINZ/BayofPlenty/LINZ20212022/bay-of-plenty-02m-rural-aerial-photos-index-tiles-2021-2022/bay-of-plenty-02m-rural-aerial-photos-index-tiles-2021-2022.shp, Arch...",2022-01-02,02/01/2022,20220102,FLOWN,100.000000,0.2


In [93]:
with pd.option_context("display.max_rows", None):
  display(LDS[LDS.match_score < 100])

,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
68,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
231,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
178,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
131,Retrolens/Canterbury/OpihiRiver_South/Shorelines/OpihiRiver_South_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
286,Retrolens/Canterbury/RakaiaSouth/Shorelines/RakaiaSouth_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
320,Retrolens/Canterbury/OpihiRiver_RangitataRiver/Shorelines/OpihiRiver_RangitataRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
248,Retrolens/Canterbury/OpihiRiver_OrariRiver/Shorelines/OpihiRiver_OrariRiver_02DEC2020.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2020-12-02,02/12/2020,14/02/17 to 09/03/17,FLOWN,41.538462,0.3
433,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
464,Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_14FEB2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-02-28,28/02/2017,14/02/17 to 09/03/17,FLOWN,42.750000,0.3
51,Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_20FEB2023.shp,[SpatialData/Mosaics/Footprint shapefiles/Hawkes Bay Footprints/hawkes-bay-03m-rural-aerial-photos-index-tiles-2019-2020.shp],2023-02-20,20/02/2023,06/03/2019,FLOWN,44.444444,0.3


In [94]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv(prefix + "Nick/LDS_matches.csv", index=False)